<a href="https://colab.research.google.com/github/mmikanguyen/DS4200Project/blob/main/DataPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
!pip install liac-arff
import pandas as pd
import arff
with open('dataset.arff', 'r') as f:
  data = arff.load(f)
df = pd.DataFrame(data['data'])

In [85]:
df.columns = ["date","quarter","department","day","team","targeted_productivity","smv","wip",
              "over_time","incentive","idle_time","idle_men","style_changes", "num_workers", "actual_productivity"]

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date                   1197 non-null   object 
 1   quarter                1197 non-null   object 
 2   department             1197 non-null   object 
 3   day                    1197 non-null   object 
 4   team                   1197 non-null   int64  
 5   targeted_productivity  1197 non-null   float64
 6   smv                    1197 non-null   float64
 7   wip                    691 non-null    float64
 8   over_time              1197 non-null   int64  
 9   incentive              1197 non-null   int64  
 10  idle_time              1197 non-null   float64
 11  idle_men               1197 non-null   int64  
 12  style_changes          1197 non-null   int64  
 13  num_workers            1197 non-null   float64
 14  actual_productivity    1197 non-null   float64
dtypes: f

In [87]:
df.describe()
# wip column has missing vals (506 missing vals)

,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,style_changes,num_workers,actual_productivity
count,1197.000000,1197.000000,1197.000000,691.000000,1197.000000,1197.000000,1197.000000,1197.000000,1197.000000,1197.000000,1197.000000
mean,6.426901,0.729632,15.062172,1190.465991,4567.460317,38.210526,0.730159,0.369256,0.150376,34.609858,0.735091
std,3.463963,0.097891,10.943219,1837.455001,3348.823563,160.182643,12.709757,3.268987,0.427848,22.197687,0.174488
min,1.000000,0.070000,2.900000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.233705
25%,3.000000,0.700000,3.940000,774.500000,1440.000000,0.000000,0.000000,0.000000,0.000000,9.000000,0.650307
50%,6.000000,0.750000,15.260000,1039.000000,3960.000000,0.000000,0.000000,0.000000,0.000000,34.000000,0.773333
75%,9.000000,0.800000,24.260000,1252.500000,6960.000000,50.000000,0.000000,0.000000,0.000000,57.000000,0.850253
max,12.000000,0.800000,54.560000,23122.000000,25920.000000,3600.000000,300.000000,45.000000,2.000000,89.000000,1.120437


In [88]:
df.head()

,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,style_changes,num_workers,actual_productivity
0,1/1/2015,Quarter1,sweing,Thursday,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725
1,1/1/2015,Quarter1,finishing,Thursday,1,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.886500
2,1/1/2015,Quarter1,sweing,Thursday,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
3,1/1/2015,Quarter1,sweing,Thursday,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
4,1/1/2015,Quarter1,sweing,Thursday,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382


In [89]:
df.isnull().sum()

,0
date,0
quarter,0
department,0
day,0
team,0
targeted_productivity,0
smv,0
wip,506
over_time,0
incentive,0


In [90]:
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day_of_month'] = df['date'].dt.day

In [91]:
# Handled missing vals in wip by filling NaN vals with the mean after grouping by team number
df['wip'] = df.groupby('team')['wip'].transform(lambda x: x.fillna(x.mean()))
df.isnull().sum()

,0
date,0
quarter,0
department,0
day,0
team,0
targeted_productivity,0
smv,0
wip,0
over_time,0
incentive,0


In [92]:
df['productivity_gap'] = df['targeted_productivity'] - df['actual_productivity']
df['productivity_ratio'] = df['actual_productivity'] / df['targeted_productivity']

df['overtime_per_worker'] = df['over_time'] / df['num_workers']
df['incentive_per_worker'] = df['incentive'] / df['num_workers']
df['idle_time_per_worker'] = df['idle_time'] / df['num_workers']
df['style_changes_per_worker'] = df['style_changes'] / df['num_workers']
df['idle_time_ratio'] = df['idle_time'] / (df['over_time'] + df['smv'])
df['low_productivity_flag'] = df['actual_productivity'] < (0.7 * df['targeted_productivity'])
df['worker_efficiency'] = df['actual_productivity'] / (df['num_workers'] * df['smv'])
df

,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,...,day_of_month,productivity_gap,productivity_ratio,overtime_per_worker,incentive_per_worker,idle_time_per_worker,style_changes_per_worker,idle_time_ratio,low_productivity_flag,worker_efficiency
0,2015-01-01,Quarter1,sweing,Thursday,8,0.80,26.16,1108.000000,7080,98,...,1,-0.140725,1.175907,120.000000,1.661017,0.0,0.0,0.0,False,0.000609
1,2015-01-01,Quarter1,finishing,Thursday,1,0.75,3.94,1609.196429,960,0,...,1,-0.136500,1.182000,120.000000,0.000000,0.0,0.0,0.0,False,0.028125
2,2015-01-01,Quarter1,sweing,Thursday,11,0.80,11.41,968.000000,3660,50,...,1,-0.000570,1.000713,120.000000,1.639344,0.0,0.0,0.0,False,0.002300
3,2015-01-01,Quarter1,sweing,Thursday,12,0.80,11.41,968.000000,3660,50,...,1,-0.000570,1.000713,120.000000,1.639344,0.0,0.0,0.0,False,0.002300
4,2015-01-01,Quarter1,sweing,Thursday,6,0.80,25.90,1170.000000,1920,50,...,1,-0.000382,1.000477,34.285714,0.892857,0.0,0.0,0.0,False,0.000552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,2015-03-11,Quarter2,finishing,Wednesday,10,0.75,2.90,1528.333333,960,0,...,11,0.121667,0.837778,120.000000,0.000000,0.0,0.0,0.0,False,0.027083
1193,2015-03-11,Quarter2,finishing,Wednesday,8,0.70,3.90,984.375000,960,0,...,11,0.074375,0.893750,120.000000,0.000000,0.0,0.0,0.0,False,0.020052
1194,2015-03-11,Quarter2,finishing,Wednesday,7,0.65,3.90,999.509091,960,0,...,11,0.024375,0.962500,120.000000,0.000000,0.0,0.0,0.0,False,0.020052
1195,2015-03-11,Quarter2,finishing,Wednesday,9,0.75,2.90,1283.724138,1800,0,...,11,0.244111,0.674519,120.000000,0.000000,0.0,0.0,0.0,True,0.011630


In [98]:
team_metrics = df.groupby(['team', 'month']).agg({
    'actual_productivity': 'mean',
    'targeted_productivity': 'mean',
    'wip': 'sum',
    'over_time': 'mean',
    'worker_efficiency': 'mean',
    'idle_time': 'sum',
    'style_changes': 'sum',
    'num_workers': 'mean',
    'incentive': 'sum',
    'productivity_gap': 'mean',
    'productivity_ratio': 'mean',
    'overtime_per_worker': 'mean',
    'incentive_per_worker': 'mean',
    'idle_time_per_worker': 'mean',
    'style_changes_per_worker': 'mean',
    'idle_time_ratio': 'mean',
    'low_productivity_flag': 'sum'  # Assuming it’s a flag, summing could give the count of occurrences
}).reset_index()

team_metrics

,team,month,actual_productivity,targeted_productivity,wip,over_time,worker_efficiency,idle_time,style_changes,num_workers,incentive,productivity_gap,productivity_ratio,overtime_per_worker,incentive_per_worker,idle_time_per_worker,style_changes_per_worker,idle_time_ratio,low_productivity_flag
0,1,1,0.828147,0.760417,69074.517857,5845.625000,0.014386,0.0,0,34.697917,1594,-0.067730,1.089131,186.552306,0.569276,0.000000,0.000000,0.000000,1
1,1,2,0.844636,0.751351,69248.142857,3647.027027,0.008950,0.0,6,36.567568,1756,-0.093285,1.125406,123.547434,0.829346,0.000000,0.003218,0.000000,2
2,1,3,0.760406,0.705000,30642.964286,4389.000000,0.011686,0.0,0,33.050000,1543,-0.055406,1.075281,166.085330,6.501441,0.000000,0.000000,0.000000,0
3,2,1,0.795592,0.762500,58017.175439,5766.250000,0.010440,0.0,0,35.437500,1566,-0.033092,1.045966,167.081919,0.570862,0.000000,0.000000,0.000000,2
4,2,2,0.788479,0.760714,67784.614035,3462.857143,0.006915,0.0,9,37.142857,1254,-0.027764,1.025014,106.716770,0.518972,0.000000,0.003724,0.000000,5
5,2,3,0.669404,0.636842,18762.807018,2933.684211,0.011276,6.5,8,27.000000,1345,-0.032562,1.052146,130.652463,6.485380,0.007602,0.009306,0.000063,0
6,3,1,0.816654,0.750000,52430.275862,6014.651163,0.009889,0.0,0,36.918605,1393,-0.066654,1.090429,174.820725,0.573339,0.000000,0.000000,0.000000,2
7,3,2,0.770338,0.700000,58556.931034,4881.818182,0.004230,0.0,20,44.272727,1315,-0.070338,1.112248,128.868669,0.695335,0.000000,0.010460,0.000000,2
8,3,3,0.833227,0.797368,22895.637931,4787.368421,0.007110,0.0,5,37.157895,2000,-0.035858,1.044825,155.873531,6.830070,0.000000,0.004569,0.000000,0
9,4,1,0.814281,0.715306,51340.593220,6726.122449,0.007099,0.0,0,37.795918,1345,-0.098975,1.204253,198.685884,0.482166,0.000000,0.000000,0.000000,1


In [99]:
dept_metrics = df.groupby(['department', 'month']).agg({
    'actual_productivity': 'mean',
    'targeted_productivity': 'mean',
    'wip': 'sum',
    'over_time': 'sum',
    'worker_efficiency': 'mean',
    'idle_time': 'sum',
    'style_changes': 'sum',
    'num_workers': 'sum',
    'incentive': 'sum',
    'productivity_gap': 'mean',
    'productivity_ratio': 'mean',
    'overtime_per_worker': 'mean',
    'incentive_per_worker': 'mean',
    'idle_time_per_worker': 'mean',
    'style_changes_per_worker': 'mean',
    'idle_time_ratio': 'mean',
    'low_productivity_flag': 'sum'
}).reset_index()

dept_metrics

,department,month,actual_productivity,targeted_productivity,wip,over_time,worker_efficiency,idle_time,style_changes,num_workers,incentive,productivity_gap,productivity_ratio,overtime_per_worker,incentive_per_worker,idle_time_per_worker,style_changes_per_worker,idle_time_ratio,low_productivity_flag
0,finishing,1,0.436326,0.700000,968.135593,1440,0.008762,0.0,0,12.0,0,0.263674,0.623323,120.000000,0.000000,0.000000,0.000000,0.000000,1
1,finishing,2,0.726645,0.739404,182570.065764,257760,0.019477,0.0,0,1626.0,0,0.012759,1.009158,158.145695,0.000000,0.000000,0.000000,0.000000,27
2,finishing,3,0.719962,0.719072,118344.106759,187140,0.020686,0.0,0,951.0,15000,-0.000890,1.013729,188.041237,14.845361,0.000000,0.000000,0.000000,14
3,finishing,1,0.783512,0.743162,281830.130365,492660,0.023789,0.0,0,2340.0,0,-0.040350,1.080588,197.231493,0.000000,0.000000,0.000000,0.000000,22
4,finishing,2,0.767612,0.743478,28562.272604,31080,0.018719,0.0,0,259.0,0,-0.024133,1.025814,120.000000,0.000000,0.000000,0.000000,0.000000,4
5,sweing,1,0.735038,0.726221,328252.000000,2543190,0.000769,0.0,0,16192.5,15315,-0.008817,1.011806,158.608274,0.982576,0.000000,0.000000,0.000000,12
6,sweing,2,0.708820,0.723123,386160.000000,1338920,0.000698,856.0,149,14098.5,10827,0.014303,0.997688,95.789232,0.796872,0.055780,0.009901,0.003374,27
7,sweing,3,0.718102,0.720000,108200.000000,615060,0.000710,18.0,31,5949.0,4596,0.001898,0.997101,103.935948,0.800038,0.002981,0.005475,0.003039,4


In [95]:
df.columns

Index(['date', 'quarter', 'department', 'day', 'team', 'targeted_productivity',
       'smv', 'wip', 'over_time', 'incentive', 'idle_time', 'idle_men',
       'style_changes', 'num_workers', 'actual_productivity', 'year', 'month',
       'day_of_month', 'productivity_gap', 'productivity_ratio',
       'overtime_per_worker', 'incentive_per_worker', 'idle_time_per_worker',
       'style_changes_per_worker', 'idle_time_ratio', 'low_productivity_flag',
       'worker_efficiency'],
      dtype='object')